In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import nltk

In [2]:
data = pd.read_csv(r'C:\Users\user\Downloads\IMDB Dataset.csv')

In [3]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
# Afficher le nombre de lignes du DataFrame
nombre_de_lignes = data.shape[0]
print("Nombre de lignes du DataFrame :", nombre_de_lignes)


Nombre de lignes du DataFrame : 50000


In [5]:
import pandas as pd

# Supprimer les dernières n lignes
n =45000  # Nombre de dernières lignes à supprimer
data.drop(data.tail(n).index, inplace=True)

In [6]:
# Afficher le nombre de lignes du DataFrame
nombre_de_lignes = data.shape[0]
print("Nombre de lignes du DataFrame :", nombre_de_lignes)


Nombre de lignes du DataFrame : 5000


In [7]:
import pandas as pd
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import spacy

In [8]:
nltk.download('stopwords')
nltk.download('punkt')
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
from nltk.tokenize import word_tokenize

# Initialisation du tokenizer, des stop words et du stemmer
tokenizer = nltk.RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

# Expression régulière pour le nettoyage des reviews
reg = re.compile("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(w+://\S+)")

# Fonction pour le prétraitement du texte avec stemming et nettoyage
def preprocess_text(text):
    # Suppression des mentions, des hashtags, des caractères non alphanumériques et des URLs
    cleaned_text = reg.sub(" ", text)

    # Tokenisation
    tokens = word_tokenize(cleaned_text)

    # Suppression des stop words et stemming
    filtered_tokens = [stemmer.stem(token) for token in tokens
                       if token.lower() not in stop_words and len(token) > 1]

    return filtered_tokens


# Appliquer le prétraitement sur la colonne 'review'
data['Cleanedreview'] = data['review'].apply(lambda x: ' '.join(preprocess_text(x)))


# Supprimer les lignes vides ou contenant uniquement des stopwords
data = data[data['Cleanedreview'].str.strip() != '']

# Afficher les premières lignes du jeu de données avec les tweets nettoyés
print(data[['review', 'Cleanedreview']].head())
print(data)


                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                       Cleanedreview  
0  one review mention watch oz episod hook right ...  
1  wonder littl product br br film techniqu unass...  
2  thought wonder way spend time hot summer weeke...  
3  basic famili littl boy jake think zombi closet...  
4  petter mattei love time money visual stun film...  
                                                 review sentiment  \
0     One of the other reviewers has mentioned that ...  positive   
1     A wonderful little production. <br /><br />The...  positive   
2     I thought this was a wonderful way to spend ti...  positive   
3     Basically there's a family where a little boy ...  

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

# Définir le vectoriseur TF-IDF avec les paramètres appropriés
vectorizer = TfidfVectorizer(stop_words=None)

# Transformer les données textuelles en vecteurs TF-IDF
X_tfidf = vectorizer.fit_transform(data["Cleanedreview"])

# Convertir la matrice TF-IDF en une matrice creuse CSR
tfidf_csr = csr_matrix(X_tfidf)

# Créer un DataFrame à partir de la matrice creuse
tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_csr,
                                             columns=vectorizer.get_feature_names_out())

# Afficher les premières lignes du DataFrame
print(tfidf_df.head())


    00  000  007  00am   01  01pm   02   04   05   06  ...  zucker  zudina  \
0  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...     0.0     0.0   
1  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...     0.0     0.0   
2  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...     0.0     0.0   
3  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...     0.0     0.0   
4  0.0  0.0  0.0   0.0  0.0   0.0  0.0  0.0  0.0  0.0  ...     0.0     0.0   

   zue  zulu  zuniga  zurich  zwart  zwick  zzzzip  zzzzzzzzzzzzzzzzzz  
0  0.0   0.0     0.0     0.0    0.0    0.0     0.0                 0.0  
1  0.0   0.0     0.0     0.0    0.0    0.0     0.0                 0.0  
2  0.0   0.0     0.0     0.0    0.0    0.0     0.0                 0.0  
3  0.0   0.0     0.0     0.0    0.0    0.0     0.0                 0.0  
4  0.0   0.0     0.0     0.0    0.0    0.0     0.0                 0.0  

[5 rows x 26546 columns]


In [13]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Les étiquettes
y = data['sentiment']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data['review'], y, test_size=0.3, random_state=42)

# Créer la pipeline
clf = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Transformation des textes en vecteurs TF-IDF
    ('clf', LogisticRegression())  # Modèle de régression logistique
])

# Entraîner le modèle
clf.fit(X_train, y_train)

# Prédictions du modèle sur l'ensemble de test
y_pred = clf.predict(X_test)

# Évaluer la performance du modèle
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Affichage des résultats
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

# Sauvegarder le modèle et le vectorizer
with open('sentiment_model.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)


Accuracy: 0.87
Precision: 0.8708685572875183
Recall: 0.87
F1-score: 0.8700627230380984


In [14]:
clf.predict(['Wow, this is amazing film'])


array(['positive'], dtype=object)

In [15]:
clf.predict(['Wow, this sucks'])


array(['negative'], dtype=object)

In [17]:
import pickle

pickle.dump(clf, open('sentiment_analysis_movies.pkl', 'wb'))

In [23]:
clf.predict(['i like so mush the film'])


array(['positive'], dtype=object)

In [19]:
clf.predict(['the film is sucks'])


array(['negative'], dtype=object)

In [20]:
clf.predict(['Wow, this is amazing film'])


array(['positive'], dtype=object)

In [27]:
clf.predict(['i don t like the film'])


array(['negative'], dtype=object)

In [28]:
clf.predict(['the film is so bad'])


array(['negative'], dtype=object)

In [29]:
clf.predict(['the film is so good'])


array(['positive'], dtype=object)